In [1]:
%matplotlib widget
import numpy as np
import glob, itertools, os

from alvra_tools.channels import *
from alvra_tools.XES_functions import *
from alvra_tools.utils import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
pgroup='p20222'
runlist = [49]
#runlist1 = np.arange(420, 425, 1)

#runlist = np.concatenate((runlist1, runlist2, runlist3, runlist4))
print ('{} runs: {}'.format(len(runlist),runlist))

#############################################################
jsonlist = []
for run in runlist:
    file = glob.glob('/sf/alvra/data/{}/work/reduced_data/no_rounding/*{:04d}*/meta/scan.json'.format(pgroup, run))
    jsonlist.append(file)
jsonlist = itertools.chain.from_iterable(jsonlist)
jsonlist = sorted(jsonlist, key = os.path.getmtime)
jsonlist

1 runs: [49]


['/sf/alvra/data/p20222/work/reduced_data/no_rounding/run0049-FebpyCN4_H2O_timescan_001/meta/scan.json']

In [28]:
threshold_low = 3
threshold_high = 50
nshots = None
nsteps = None

reducedir='../Claudio/Reduced_TT/'

TT = TT_PSEN126
Timezero_mm = 10.02694
channel_delay_motor = channel_delay_topas

#####################################################################################
from sfdata import SFScanInfo

for json in jsonlist:
    scan = SFScanInfo(json)
    try:
        Timezero_mm = get_timezero_NBS(json)
        flag = True
    except:
        Timezero_mm = Timezero_mm
        
    Delays_fs_scan, Delays_corr_scan, \
    spectra_ROIs_ON, spectra_ROIs_OFF, \
    Delay_fs, Delay_mm = \
    XES_delayscan_TT_reduced(scan, pgroup, TT, channel_delay_motor, Timezero_mm, threshold_low, threshold_high, nshots, nsteps=nsteps)
    
    print ('------------------------------')
    if flag:
        print ('Timezero found in file = {} mm'.format(Timezero_mm))
    else:
        print ('Timezero = {} mm'.format(Timezero_mm))

    if scan.parameters['units'] == ['nS']:
        Delay_fs = - scan.values
    #    Delay_fs = Delay_fs - 0.17
    else:
        Delay_fs = scan.values
        Delay_fs = Delay_fs - mm2fs(Timezero_mm, 0)
        
    run_name = json.split('/')[-3]
    os.makedirs(reducedir+run_name, exist_ok=True)

    #np.save(reducedir+run_name+'/delays_corr.npy', Delays_corr_scan)
    #np.save(reducedir+run_name+'/delays_stage.npy', Delays_fs_scan)
    #np.save(reducedir+run_name+'/Delay_readbacks.npy', Delay_fs)

    #for i, spectrum in enumerate([spectrum_roi1_ON, spectrum_roi2_ON, spectrum_roi3_ON, spectrum_roi4_ON]):
    #    np.save(reducedir+run_name+'/spectrum_roi{}_ON.npy'.format(i+1), spectrum)

    #for i, spectrum in enumerate([spectrum_roi1_OFF, spectrum_roi2_OFF, spectrum_roi3_OFF, spectrum_roi4_OFF]):
    #    np.save(reducedir+run_name+'/spectrum_roi{}_OFF.npy'.format(i+1), spectrum)
    
    print ('Processed run {}'.format(json.split('/')[-3]))
    

It took 2.23 seconds to process this file

Job done! It took 96.23 seconds to process 31 file(s)
------------------------------
Timezero found in file = 100.01392000004499 mm
Processed run run0049-FebpyCN4_H2O_timescan_001


In [51]:
spectrum_ROI_ON = len(spectra_ROIs_ON)*[0]
for i,key in enumerate((spectra_ROIs_ON.keys())):
    spectrum_ROI_ON[i] = spectra_ROIs_ON[key]
    spectrum_ROI_ON[i] = np.asarray(list(itertools.chain.from_iterable(spectrum_ROI_ON[i])))

In [53]:
binsize = 50

binList = np.arange(Delays_fs_scan[0] - 100, Delays_fs_scan[-1] + 100, binsize)
bin_centres = (binList[:-1] + binList[1:])/2
Delay_fs_TT = np.arange(Delays_fs_scan[0] -100 + binsize/2, Delays_fs_scan[-1] + 100 - binsize/2, binsize)

for roi in range(len(spectrum_ROI_ON)):
    spectrum_ROI_on_rebin[roi] = np.zeros((len(bin_centres), np.shape(spectrum_ROI_on_rebin[roi])[1]))
    

NameError: name 'spectrum_ROI_on_rebin' is not defined

In [52]:
len(spectrum_ROI_ON)

8

In [4]:
spectrum_roi1_ON = spectra_ROIs_ON['roi_0']
spectrum_roi2_ON = spectra_ROIs_ON['roi_1']



In [4]:
roi_array = [spectrum_roi1_ON, spectrum_roi1_OFF, 
             spectrum_roi2_ON, spectrum_roi2_OFF,
             spectrum_roi3_ON, spectrum_roi3_OFF,
             spectrum_roi4_ON, spectrum_roi4_OFF,]

for index in range(len(roi_array)):
    roi_array[index] = np.asarray(list(itertools.chain.from_iterable(roi_array[index])))
    print (np.shape(roi_array[index]))

spectrum_roi1_ON = spectra_ROIs_ON['roi_0']

In [7]:
np.shape(spectrum_roi1_ON)

(13304, 120)

In [6]:
spectrum_roi1_ON = np.asarray(list(itertools.chain.from_iterable(spectrum_roi1_ON)))